<span type="title">Spring MVC</span> | <span type="update">2018-10-25</span> - Version <span type="version">1.1</span>
    
    
<span type="intro"><p class="card-text">本章主要介绍了 Spring MVC 的基本使用。首先介绍了 Spring MVC 的使用概览，包括 MVC 文件配置、RequestMapping 注解的使用、ModelAndView 对象的处理、基本的 Model 绑定、视图和视图解析器。</p><p class="card-text">在第二部分，介绍了基于 Spring MVC 的 Restful 风格的 CRUD，重点介绍了 PUT 和 DELETE 的实现。</p><p class="card-text">在第三部分，讲解了高级的 Spring MVC 数据绑定，包括格式转换服务、数据格式化、数据校验。之后介绍了对于 HttpMessageConverter 的使用：处理 JSON，实现简单的上传下载，在这部分结尾介绍了基于 MultipartResolver 的文件上传。</p><p class="card-text">在第四部分，介绍了 Spring MVC 的拦截器以及拦截顺序、四种常见的异常处理器，包括异常映射处理、简单Mapping处理、自定义页面跳转处理和 HTTP 状态错误跳转处理。</p><p class="card-text">在第五部分，介绍了 Spring MVC 的整体运行流程，包括 HandlerMapping 和 HandlerAdapter，ViewResolver 和 View，拦截器在整个生命周期的调用情况。在本文的最后，对 Spring 整合 Sping MVC 提出了建议，将 MVC 看作 Spring IOC 的一个子容器，通过 web.xml 的监听器来加载 Spring IOC bean。尤其重要的是，注意包扫描的问题，不要因为重复扫描浪费资源。</p></span>

# Spring MVC 概述

Spring MVC 是一套用来代替 Struts 的 Servlet 框架。相比较传统的 Servlet 技术，基于 Struts 的框架设计模式采用一个 Super Servlet 对所有的 URL 请求进行转发和分派，然后再交给子 Servlet 进行请求，之后由 JSP 等模板技术完成渲染工作。这套框架很好的避免了大工程需要繁琐的配置 web.xml 文件、同时能够对于 URL 请求进行更轻松的解析和处理。Spring MVC 基于 Spring IOC 容器，是 Struts 的替代产品，主要用来进行 Controller 层的控制，完成 View 层和 Model 层之间的交互。

其基本作用原理是：使用 Spring 内置的 Servlet 拦截所有请求，交给自家的 Handler 完成处理，之后再委托 JSP 等模板引擎进行渲染。需要配置 web.xml 文件完成拦截和监听，配置 spring-mvc.xml 等配置文件来具体定义 Handler 的查找路径、模板引擎的处理器等等。

Spring MVC 相比较 Struts 的优点在于：天生和 Spring 框架集成，支持 Restful 风格的开发，从 URL 到映射的处理十分方便，数据验证、格式化和绑定功能强大、好用，对于异常的处理支持较好，对于静态资源文件提供了支持。

下面将会从基本配置（使用概览）、Controller 类方法的方法注解（负责URL映射）、方法入参（获取 URL 参数，Param 参数，Servlet API 对象，数据绑定对象，ModelAndView 的 ModelMap 对象）、方法返回值（ModelAndView、String 的各种 ViewResolver：InternalResourceViewResolver、BeanNamedViewResolver 和 View：InternalResourceView，JSTLView，自定义View，RedirectView）三个层面进行详细介绍。

## MVC 使用概览

**WEB.XML配置**

Web.xml 文件需要配置如下的 Servlet，完成拦截，需要指明类为 DispatcherServlet，此外，需要传入 Spring MVC 的 xml 格式配置，放置在 Servlet 的上下文中，名称为 ContextConfigLocation，如果这里没有配置，则默认寻找 `/WEB-INF/{serlvet-name}-servlet.xml` 文件作为配置。

```xml
<servlet>
    <servlet-name>springDispatcherServlet</servlet-name>
    <servlet-class>org.springframework.web.servlet.DispatcherServlet</servlet-class>
    <init-param>
        <param-name>contextConfigLocation</param-name>
        <param-value>springmvc.cfg.xml</param-value>
    </init-param>
</servlet>
<servlet-mapping>
    <servlet-name>springDispatcherServlet</servlet-name>
    <url-pattern>/</url-pattern>
</servlet-mapping>
```

为了支持 Restful 风格 URL 格式，推荐添加如下过滤器, HiddenHttpMethodFilter 过滤器可以对请求进行过滤，根据隐藏字段，将 POST, GET 请求转换成 POST, GET, PUT, DELETE 四种原生的 HTTP 方法。

```xml
<filter>
    <filter-name>methodFilter</filter-name>
    <filter-class>org.springframework.web.filter.HiddenHttpMethodFilter</filter-class>
</filter>
<filter-mapping>
    <filter-name>methodFilter</filter-name>
    <url-pattern>/*</url-pattern>
</filter-mapping>
```
Ps. 使用 jsp 设置组来去除空白字符需要注意，映射为 `*.jsp` 而不要写成 `/*`,否则所有 serlvet 均会失效。

PPs. 这里的 Spring Servlet 的 URL 是 `/` 最好，而不要是 `/*`, 否则将会自动处理任意 .jsp 文件，当然，如果你在 SPRING-MVC 的配置文件中指定了如果处理不了，交还应用服务器进行处理这一选项 `default-serlvet` 的话，怎样写无所谓。

**SPRING-SERVLET.XML配置**

一个基本的 springmvc.cfg.xml 配置文件如下：

```xml
<?xml version="1.0" encoding="UTF-8"?>
<beans xmlns="http://www.springframework.org/schema/beans"
       xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
       xmlns:beans="http://www.springframework.org/schema/beans"
       xmlns:context="http://www.springframework.org/schema/context"
       xmlns:mvc="http://www.springframework.org/schema/mvc"
       xsi:schemaLocation="http://www.springframework.org/schema/beans
       http://www.springframework.org/schema/beans/spring-beans.xsd
       http://www.springframework.org/schema/context
       http://www.springframework.org/schema/context/spring-context.xsd
       http://www.springframework.org/schema/mvc
       http://www.springframework.org/schema/mvc/spring-mvc.xsd
">
    <context:component-scan base-package="com.mazhangjing.mvc.hello"/>
    <mvc:annotation-driven />
    <bean class="org.springframework.web.servlet.view.InternalResourceViewResolver" 
          p:prefix="/WEB-INF/views/" p:suffix=".jsp"/>

    <!--或者可以不经过 Handler，直接完成 View-Controller 和 View 的对应。此种情况下需要添加 mvc 注解驱动-->
    <mvc:view-controller path="/success" view-name="success" />

</beans>
```

注意这里面较为重要的一些设置：

- 【基于注解扫描的 Handler】：启用 context:component-scan 和 mvc:annotation-driven, 这两个选项能够保证指定位置的包中的注解能够被被配置为 bean，并且能够作为 Handler 进行映射。

- 【ViewResolver 页面解析器】：一般使用的是 InternalResourceViewResolver，传入 suffix 配置 .jsp 作为后缀，perfix 作为前缀，之后，你可以在 Handler 中返回字符串，然后 ViewResolver 会自动进行前缀+字符串+后缀的拼接，然后找到指定位置的 jsp 并进行解析。

    - 尤其需要注意，此处的 perfix 前缀需要以 `/` 结尾，否则拼接出来的地址会是错误的！！！

    - 除了使用 Handler 进行映射，也可以直接使用 mvc:view-controller 映射指定 URL 到指定的 view，这里的 view-name 会拼接 ViewResolver 找到对应位置的视图引擎自动解析并且返回 HttpResponse 流。

**CONTROLLER HANDLER 配置**

```java
@Controller
public class HelloWorld {
    @RequestMapping(value = "/helloworld",method = RequestMethod.GET)
    public String sayHello() {
        System.out.println("Controller get sayHello request from SpringMVC Servlet");
        return "success";
    }
}
```

为了让 Spring 添加 bean，需要添加 @Controller 注解，为了指定映射的 URL，需要指定类或者方法的 @RequestMapping 注解，提供 URL 和处理的 Http 方法即可。签名随意，返回值为 String 会自动被视图解析器拼接，找到视图层，进行返回。之后访问 `localhost:port/helloworld` 即可调用此处的 Handler 进行处理。

**Spring MVC 具体工作流程**

1、【找到 DispatcherServlet】

  启动 TOMCAT 后会自动在 web.xml 找到 Servlet 进行请求响应。而在 XML 中，我们使用了 Spring 内置的 servlet
  处理所有的发送请求，这个 servlet class 是 org.springframework.web.servlet.DispatcherServlet. 相当于 Structs 的
  巨型 Dispatcher。此 Servlet 默认会处理 WBE-INF 下的 servlet_name + "-servlet.xml" 结尾的描述文件，用作配置。此外，
  也可以传递 configLocation 参数为指定 classpath 或者 文件路径下的 xml Spring 配置文件。
 
2、【找到 HandlerMapping】
  
  Spring 配置文件规定了这个 Servlet 如何处理相应。区别于直接在 web.xml 写 URL 转发规则， Spring 使用注解来决定
  转发，因此启用 component-scan 注解扫描，Spring 会自动扫描 @Controller 注解类型 以及 其内部的 @RequestMapping 方法
  用来进行转发，@RequestMapping 需要提供 value 和 method 以声明自身可以处理何种转发。
 
  1-2步即是 TOMCAT 找到 Spring Dispatcher ， 然后 SD 通过 xml 配置找到匹配注解的 RequestMapping 方法进行转发。这里完成了
  传统 Servlet 从 URL 到 Java 类方法的过程。
 
3、【找到 ViewResolver】
  
  接下来可以直接像 Servlet 一样分派一个 JSP 处理 URL 请求，这里使用的是一个更好玩的类，叫做 org.springframework.web.
  servlet.view.InternalResourceViewResolver。这个 ViewResolver 在 Spring xml 中进行了声明，提供了 prefix 和 suffix
  属性，这里不通过交给 jsp 进行分派，而是返回一个 String，因此交给了此 ViewResolver 获得指定 prefix + String value + suffix
  位置的 jsp 文件进行自动。最后由此处的 jsp 处理视图相应。
  这里的好处是，可以通过替换 ViewResolver 来达到前后端分离，或直接处理前端内容，更加灵活。此外，对于超级多的 jsp 页面逻辑支持也更好。
 

## RequestMapping 详解

RequestMapping 用来解析 URL 到指定的 Handler，除了修饰方法以外，还可以修饰类。使用基于类 + 基于方法的 RequestMapping 可以用来分级的解析地址：`/springmvc/abc` 。这样的话，类注解对应高一级的地址层级，而方法注解对应稍低一级的地址层级，这样比直接在 web.xml 根据 * 和 后缀过滤  URL 方便很多。

```java
@Controller
@RequestMapping("/springmvc")
public class MVCTest {
    private static final String SUCCESS = "success";
    @RequestMapping("/abc") public String testMapping() { return SUCCESS; }
}
```
**过滤方法、地址匹配**

可以指定 Mapping 的 method 参数指定解析的 HTTP 方法，如果没有对应的方法被 Handle，则报错。

可以通过星号通配符匹配任意字符：其中 `*` 表示任意字符（单层路径）， `？` 表示一个字符， `**` 表示多层路径

```java
@RequestMapping(value = "/testMethod",method = RequestMethod.POST)
public String testMethod() { return SUCCESS; }
@RequestMapping("/testAnt*") public String testAnt() { return SUCCESS; }
```

**过滤URL参数、请求头**

可以通过 params 指定 URL 参数的匹配，可选 有/无某参数，参数等于、不等于某值 这四种情况。

可以通过 headers 指定针对仅含有某请求头匹配情况进行相应，比如下面仅对 keep-alive 的 Connection 的 headers，URL 参数包含 ursename，passwd，school，并且 passwd 不等于 10，school 必须等于 ccnu，没有 superuser 这个参数的 URL 进行相应，比如: `testParameter/?username=tom&passwd=233&school=ccnu&nosuperuser`

```java
@RequestMapping(
        value = "testParameter",
        params = {"username","passwd!=10","school=ccnu","!superuser"},
        headers = {"Connection: keep-alive"}
)
public String testParam() {
    return SUCCESS;
}
```

**从 URL 获取变量**

我们可以从 URL 中获得某些数据，而不是从 URL 的参数中获取：

```java
@RequestMapping("/testPathVar/{id}")
public String testPathVar(@PathVariable("id") Integer id) { System.out.println("id is " + id); return SUCCESS; }
```

这样的话，id 会被作为 String 从 URL 上匹配并且获取，然后传递给签名入参的 Interger id（进行了类型自动转换），然后获取了此 URL 上的信息。

为了做到从 URL 上提取数据，作为入参传递，必须指定此入参为 `@PathVariable('name_in_url')`，传入的参数为其在 URL 上双括号包裹起来的名称。注意，这种情况下，`/testPathVar/` 将不会匹配此条记录。

**RESTFUL URL 支持**

PathVariable 为 RESTFUL 风格的 URL 提供了支持，具体如下：RESTFUL 风格通过 URL + HTTP METHOD 达到 CRUD 的效果

 * 新增： /order POST   add
 * 修改： /order/1 PUT  update?id=1
 * 获取： /order/1 GET  get?id=1
 * 删除： /order/1 DELETE  delete?id=1
 
因为浏览器一般不支持 PUT 和 DELETE，因此需要配置 Servlet Listener，并且对于所有 URL 请求都启用`org.springframework.web.filter.HiddenHttpMethodFilter` 以将带有隐含域的 POST 请求转换为 DELETE 和 PUT，具体做法如下：

对于 PUT 和 delete 请求，在 form 需要添加一个 hidden type 的 name 为 _method 的输入，value 为 DELETE/PUT。

具体的 HTML 为：

```html
POST 新建对象: 
<form method="post" action="/springmvc/testRest">
    <input type="submit" value="post it" />
</form>
GET 获取某对象: 
<h1><a href="/springmvc/testRest/1"> Test REST Get </a> </h1>
PUT 更新某对象:
<form method="post" action="/springmvc/testRest/1">
    <input type="hidden" name="_method" value="PUT"/>
    <input type="submit" value="put it" />
</form>
DELETE 删除某对象:
<form method="post" action="/springmvc/testRest/1">
    <input type="hidden" name="_method" value="DELETE"/>
    <input type="submit" value="delete it" />
</form>
```
此外，PUT 和 DELETE 在 Dispatcher Mapping 中需要指定 method = `RequestMethod.PUT/DELETE`. 对于 get、put、delete 需要启用 {} 和 @PathVariable 注解。

```java
@RequestMapping(value = "/testRest/{id}",method = RequestMethod.GET)
public String testRestGET(@PathVariable Integer id) {
    System.out.println("rest GET id " + id);
    return SUCCESS;
}

@RequestMapping(value = "/testRest",method = RequestMethod.POST)
public String testRestPOST() {
    System.out.println("rest POST");
    return SUCCESS;
}

@RequestMapping(value = "/testRest/{id}",method = RequestMethod.DELETE)
public String testRestDEL(@PathVariable Integer id) {
    System.out.println("rest DEL id " + id);
    return SUCCESS;
}

@RequestMapping(value = "/testRest/{id}",method = RequestMethod.PUT)
public String testRestPUT(@PathVariable Integer id) {
    System.out.println("rest PUT id " + id);
    return SUCCESS;
}
```

**获取参数、Header、Cookie**

通过设置 RequestParam 来要求输入某一参数信息或者设置默认值、获取此参数信息。

通过设置 RequestHeader 来获取头部信息

通过设置 CookieValue 来绑定/获取 Cookie 值。

有时候可能没有对应的值或者记录，可以使用 required 来设置是否必须有此值/记录才能解析，使用 defaultValue 来设置如果没有，且 required = false 的情况下使用的替换值。

需要注意，获取的值必须和入参参数的类型匹配，或者起码可以转换，如果不能转换，则报错。

```java
@RequestMapping(value = "/testPara")
public String testPara(
        @RequestParam(value = "username", required = false, defaultValue = "Guest") String username,
        @RequestHeader(value = "Accept-Language", required = false) String al,
        @CookieValue(value = "JSESSIONID",required = false) String sessionId) {
    System.out.println("username is " + username);
    System.out.println("Accept language is " +al);
    System.out.println("session id is " +sessionId);
    return SUCCESS;
}
```

**获取 Servlet 原生 API**

下面展示了入参使用传统 ServletAPI 的情景，将这些类型放置在入参处即可，不需要加注解。Spring MVC 具体支持 HSRequest/Response、Session、Input/OutputStream、Reader/Writer、Principal 这些原生 API 作为入参传递。

注意，返回值可以不为 String，不交给视图解析器，而是直接向输出 write 数据。

```java
@RequestMapping("/testServlet")
public void testServletAPI(HttpServletRequest request, HttpServletResponse response,
                             Writer out) throws IOException {
    System.out.println(request + "" +  response);
    out.write("Hello from Servlet API");
}
```

## ModelAndView 对象

**手动返回 ModelAndView 对象传递 RequestAttr**

一般来说，通过 String 返回找到 view 层即可完成响应。但是，如果需要向 view 层写数据，就需要使用 modelAndView 对象作为返回。返回 String 则视图解析器返回视图 View，也可以返回 ModelAndView 对象，此对象包括了模型和 View 层信息，可以向 View 层传递数据。区别于直接返回 String 并直接返回 view 层，视图解析器将会将 modelAndView 对象中找到对应的 view 层，将 model 的键值对解析到 view 层的 rS 域中，然后再返回 view 层。


```java
//Controller.java
@RequestMapping("/testModelAndView") public ModelAndView testModelAndView() {
    String viewName = SUCCESS;
    ModelAndView modelAndView = new ModelAndView(viewName);
    modelAndView.addObject("time",new Date());
    return modelAndView;
}
//success.jsp
${requestScope.time}
```

如上所示，我们可以在 ModelAndView 对象中添加 key-value，然后这些 key-value 会被自动 set 到 Request Attribute 中，在 View 层可以直接取出。需要注意，手动返回 MAV 的时候，需要设置到何处找到 view 层的信息（一般通过构造器直接传入）。

不论 Mapping 返回的是 String，还是 ModelAndView，都会被装配成 ModelAndView 对象，其包含了逻辑名和模型对象的视图。ViewResolver 用来将 此对象 变为真正的 View 视图对象。视图可以是 JSP，可以是 Excel、JFreeChat等。InternalResourceViewResolver 对应的是 InternalResourceView 对应的是 JSP 视图。

**使用 Map 入参传递 RequestAttr 数据**

MAV 方式过于手动且繁琐，实际情况下有一种简写方式，可以使用 Map 作为入参，然后在 Map 中放置键值对，然后从 rS 域或者直接取出。入参除了 Map 类型，还可以使用 ModelMap、Model 类型。

这种方式其实是自动的将 Map 的数据放置到了 ModelAndView 的 ModelMap 中，然后通过处理 ModelAndView 进行的处理。视图解析器对 ModelAndView 的处理方法是：通过将 MAV 中 ModelMap 的 k-v 对 设置为请求域的属性，然后去解析 View 层，将 HTML 作为结果返回。

```java
@RequestMapping("/testMap") public String testMap(Map<String, Object> map) {
    map.put("names", Arrays.asList("A","B","C"));
    return SUCCESS;
}
//success.jsp
${requestScope.names}
```

**将 RequestAttr 的 K-V对 同时添加到 SessionAttr 上**

放置在 Map 入参处的值，可以通过 rs 或者 key 直接取出。通过设置：@SessionAttributes({"user"}) 可以同时放置到 SessionAttributes 域中。此域的 value 属性仅允许设置过的 key 保存，如果没有设置，则不允许。此域的 type 属性可以允许按照 class 类型来讲 rs 域值放置到 sa 中。比如：@SessionAttributes(type={String.class}) 这样将会让所有 rs 域中 String 类型的数据放置到 SessionAttribute 上。

```java
@SessionAttributes(value = {"user"},types = {String.class})
@Controller
@RequestMapping("/springmvc")
public class MVCTest {
    ...
    @RequestMapping("/testSession")
    public String testSessionAttribute(Map<String, Object> map) {
        User user = new User(1,"corkine","123456","c@cm.cn",15);
        map.put("user",user);
        return SUCCESS;
    }
```

## Model 数据绑定

**从 FORM 到 POJO**

SpringMVC 支持直接 POST 参数到一个 POJO 中，还支持级联属性。以下是两个 POJO 类：

```java
public class User {
    private Integer id;
    private String name;
    private String password;
    private String email;
    private Integer age;
    private Address address;
    ...
}
public class Address {
    private String province;
    private String city;
    ...
}
```

注意，POJO 必须提供无参构造器和 get/set 方法。

以下是 HTML 调用，需要使用 post 方法，此外，form 的 name 需要对应 pojo 的属性名称， pojo 需要提供 set 方法，支持级联属性，比如 address.city 表示 address 对象 的 city 属性。直接在 name 中设置  'address.city' 即可。如果获取不到则设置为 null；

```html
/springmvc/testPOJO
<form method="post" action="">
    username : <input type="text" name="name" /><br>
    password : <input type="password" name="password" /><br>
    email : <input type="text" name="email" /><br>
    age : <input type="text" name="age" /><br>
    province : <input type="text" name="address.province" /><br>
    city : <input type="text" name="address.city" /><br>
    <input type="submit" value="Submit" />
</form>
```

之后直接在入参处写一个此类型的入参即可调用获得构造好的对象。注意，不要写 @RequestVariable，直接 POST 即可。这一套从 HTML 到 Bean 的方式类似于 Servlet 的 Stand Action Use-Bean。

```java
@RequestMapping("/testPOJO")
public String testPojo(@ModelAttribute("user") User user) { System.out.println(user); return SUCCESS; }
```

这里获取到的 user 会被同样放置到 requestScope 中，通过入参的 @ModelAttribute(value) 设置 requestScope 中获取此 user 的方式，比如 @ModelAttribute（value = "user"） 在  JSP 渲染页面的 ${requestScope.user} 通过这种方式获取此对象。如果不提供此注解，则默认以类名第一个字母小写作为变量放置到 requestScope 中。

```xml
/success.jsp
requestScope user: ${requestScope.user}
```

很显然的，Spring MVC 完成了从 FORM 到 POJO 的工作，那么它是如何完成的呢？

在上面我们说过，ModelAndView 对象负责用来传递 RequestAttribute 属性和 SessionAttribute 属性，并且自动完成 View 层的查找和交接工作。

这里的过程分为两步，其一是从 FORM 表单提交到构建 User 对象，到作为入参的 User 对象引用，其二是作为入参的 User 对象到 ModelAndView 对象的 ModelMap 集合，然后到 RequestAttribute 属性，以至于在 View 层能够从 requestScope 获取此对象。

**从数据库+表单到 POJO**

在一些情况下，我们并不想让表单填写所有的字段，有一些字段我们希望从数据库加载，然后只提供有限的字段以供表单修改，然后将其合并成为 POJO 作为入参传递到 Controller 的对应 Mapping 的方法中的入参中。为做到这一点，我们需要添加一个带有 @ModelAttribute 注解的方法，以完成从数据库到 FORM 调用前这一段的 User 对象的构建。

从数据库获取记录首先要从表单找到某个字段，作为查找的基础进行查找，然后获取到此记录后，再将表单剩余的字段更新此对象，最后作为入参进行设置。因此，这里需要提供一个用作查找数据的入参。找到数据后，需要将数据传递出去，这里使用的是 Map 的入参，根据前面的介绍，我们知道这里的含义是创建一个 ModelAndView，然后填入 ModelMap 的一个字段的意思。

之后，SpringMVC 会自动进行此对象根据表单的更新，然后再 Controller 的对应 Mapping 方法的入参中，使用 @ModelAttribute 注解按照 key 的名称取出即可。

一般来说，我们使用类名第一个字母小写作为 key 的名称，那样的话，这里的入参其实可以省略 @ModelAndView 注解。

```java
@ModelAttribute
public void getUser(@RequestParam(value = "id", required = false) Integer id,
                    Map<String, Object> map) {
    if (id != null) {
        User user = new User(1,"tom","123456","tom@tom.com",12);
        System.out.println("Get" + user);
        map.put("form_user",user);
    }
}
@RequestMapping("/testModelAttribute")
public String testModelAttribute(@ModelAttribute(value = "form_user") User user) {
    System.out.println("Edit " + user);
    return SUCCESS;
}
```

**内部原理详解**

具体的 ModelAttribute 调用方法如下：

首先，当 form 进行提交的时候，首先创建了一个叫做 WebDataBinder 的对象，此对象具有两个关键属性，target 和 objectName。target
用来获取模型现有对象，作为数据的入口，objName 用来指定一个字符串，用来作为更新表单数据后的数据出口标识符。

首先要获取模型数据，具体而言，先找到有 @ModelAttribute 注解的方法，将 FORM 指定字段作为参数传入，执行这段代码以更新入参的 Map 对象，并且将其放置在一个叫做 implicitModel 对象中。然后从 implicitModel 中查找对应的属性值，如果存在，ok。如果不存在，则验证当前 Handler 是否采用了 @SessionAttributes 修饰，然后从中寻找 attrName 对应的属性值，如果有此修饰符，并且有 key，但是获取不到 value，则抛出异常。如果没有此修饰符，则通过反射创建 POJO 对象。

数据的辨识名称首先如果在 Mapping 的入参 User 前没有 @ModelAttribute 修饰的话，那么我们使用类的第一个字母小写。如果有修饰的话，
使用 value 值作为 ObjName。

当得到模型的数据后，现在的 target 已经从 @ModelAttribute 的 implicitModel 中通过数据辨识名称或者 @SessionAttribute 中通关数据辨识名称获取到对象了。然后从 form 获取字段，复写 target 对象。完成之后，将数据重新传回给 implicitModel 对应字段，进而传递给 requestScope 中。而把 target 作为参数传递给目标方法的入参（Mapping 方法的 User user）。

注意，如果在 Mapping 中规定的数据标识符必须可以和 数据库取出后放置到 Map 的 Key 对上。否则的话，会创建新的 POJO 对象。

注意，如果启用了 @SessionAttribute 但是在 @MA 无法找到，在 @SA 找到但是无值 的话，抛出异常而不是新建 POJO 对象。

注意，通过 ModelAttribute 方法创建的对象最后会用 objName 作为 requestScope 的变量可供视图层调用。

## Controller 方法入参注解总结

对于所有的入参，这些入参可以在方法体内被设置后，被放置到 view 层的 requestScope 中。此外还有一些较为特殊的入参，比如

- 标记有 @PathVariable 的入参，其在方法体之前，从URL中得到了参数。


- 标记有 @RequestParam/Header/CookieValue 的入参，其在方法体之前，从请求中得到了参数/Header/Cookie。


- 作为 POST 方法的入参，POST 必定会得到一个 POJO 对象，如果入参标记了 @ModelAttribute，则传递给对应的入参。如果入参没有标记，则传递给类型匹配的入参。注意，当入参标记了 @ModelAttribute，MVC默认查找所有 @ModelAttribute 方法所返回的 Map中是否具有相同引用的 POJO，如果有，则使用此 POJO 填充字段，然后再作为入参使用。


- 如果入参是 ModelMap 或者 Map 类型，区别于别的入参整个放置到 request 域中，Map 类型自动将这些 key-value 解包，然后逐个放置到 request 域中。


- HttpRequest、Response、Session、Writer 等原生 Servlet API 作为入参，以供在方法体中调用。

## View 和 ViewResolver 视图解析


**InternalResourceView 的 JSP 转发**

不论 Mapping 返回的是 String，还是 ModelAndView，都会被装配成 ModelAndView 对象，其包含了逻辑名和模型对象的视图。ViewResolver 用来将 此对象 变为真正的 View 视图对象。视图可以是 JSP，可以是 Excel、JFreeChat等。InternalResourceViewResolver 对应的是 InternalResourceView 对应的是 JSP 视图。

视图是线程安全的，每次都会重新创建。视图规定了用来解析页面的不同策略，对于URL资源视图来说，将JSP封装成了 InternalResourceView。如果需要使用国际化功能，则启用 JstlView。此外，还有 AbstractExcelView 返回文档视图。

**JstlView 和国际化**

添加 JSTL 标签库，自动使用 JSTLView，在类路径下设置 i18n_properties 配置翻译表，通过将其配置到 messageSource 的 bean 中来指定国际化文件配置。在 JSP 页面中启用 taglib 的 fmt:message 来设置 key。

- 首先需要启用 messageSource Bean，指定翻译文件名称。

```xml
<bean class="org.springframework.context.support.ResourceBundleMessageSource" p:basename="i18n"/>
```

- 需要提供翻译文件，创建几个 i18n.properties 即可。

```
/i18n.properties
i18n.username=Username
i18n.password=Password
    
/i18n_en_US.properties
i18n.username=Username
i18n.password=Password
    
/i18n_zh_CN.properites
i18n.username=用户名
i18n.password=密码
```

- 在 JSP 中使用 fmt:message 标签，声明 key 即可根据系统语言来自动调用 JstlView 来设置界面语言的标签。

```jsp
<fmt:message key="i18n.username" />
```

**自定义 View 和 BeanNamedViewRsolver**

除了使用默认的 View，我们可以指定 BeanNamedViewResolver 来解析自定义 View。需要添加 bean，并且指定一个较高的解析优先级(数值越低越优先)：

```xml
<bean class="org.springframework.web.servlet.view.BeanNameViewResolver" p:order="100" />
```

此外，需要自定义 bean view，此 view 类需要继承自 View 接口，提供返回文档的类型，提供返回的内容，之后需要添加到 BeanFactory 中去，启用注解和注解扫描，这样的话，当我们返回一个 String，会先去找这个 String 是否对应一个 自定义 view 类，如果找不到，则换用InternalResourceViewResolver 到指定位置寻找 jsp/jstl view 进行解析。

```java
@Component("helloView")
public class HelloView implements View {

    @Override
    public String getContentType() {
        return "text/html";
    }

    @Override
    public void render(Map<String, ?> map, HttpServletRequest httpServletRequest, 
                       HttpServletResponse httpServletResponse) throws IOException {
        httpServletResponse.getWriter().println("Hello view, time: " + new Date());
    }
}
```

注意，这个 View 也是需要作为 bean 注入的，此外，需要提供 bean Name，这个 bean Name 会对应你 Handler 返回的那个字符串：

```java
@RequestMapping("/testView") public String testView() { return "helloView"; }
```

注意，自定义 View 一般需要提供 getContentType 实现返回文档类型，提供 render 实现以进行对页面的操作。

**RedirectView 跳转** 

**InternalResourceView 手动转发**

在之前的介绍中，我们已经讲过 Handler 返回 void、String 的不同情况。其中 String 会通过各种 ViewResolver 来解析视图层，比如用于 JSPView 和 JSTLView 的 InternalResourceResolver，还有用于自定义View 的 BeanNameResolver。

除此之外，还可以进行 HTTP 的链接跳转，注意，区分 forward 和 redirect，前者是分派，后者是 HTTP 跳转。当返回值为 redirect: 或者 forward: 开头，则调用 RedirectView 或者 InternalResourceView 进行重定向或者转发

```java
@RequestMapping("/testRedirect")
public String testRedirect() {
    return "redirect:/index.jsp";
    //return "forward:/index.jsp";
}
```

# CRUD示例

## Entry and Service

我们有如下持久化类以及其 Data Access Object，下面将会以此为基础创建 CRUD 示例：

```java
public class Employee {
	private Integer id;
	private String lastName;
	private String email;
	private Integer gender;
	private Date birth;
	private Float salary;
    private Department department;
    ...
}
public class Department {
	private Integer id;
	private String departmentName;
    ...
}
public interface EmployeeDao {
	public void save(Employee employee);
	public Collection<Employee> getAll();
	public Employee get(Integer id);
	public void delete(Integer id);
}
public interface DepartmentDao {
	public Collection<Department> getDepartments();
	public Department getDepartment(Integer id);
}
```

## GET 获取

在 /emps URL 请求中，展示所有的员工信息：

```java
@RequestMapping("emps")
public String list(Map<String, Object> map) {
    map.put("employees", employeeDao.getAll());
    return "list";
}
```

JSP 实现如下：

```
<c:if test="${empty requestScope.employees}">没有任何员工信息</c:if>
<c:if test="${!empty requestScope.employees}">
    <table border="1" cellpadding="10" cellspacing="0">
        <tr>
            <th>ID</th>
            <th>LastName</th>
            <th>Email</th>
            <th>Gender</th>
            <th>Deps</th>
            <th>Edit</th>
            <th>Delete</th>
        </tr>
        <c:forEach items="${requestScope.employees}" var="emp">
            <tr>
                <td>${emp.id}</td>
                <td>${emp.lastName}</td>
                <td>${emp.email}</td>
                <td>${emp.gender == 0 ? 'Female' : 'Male'}</td>
                <td>${emp.department.departmentName}</td>
                <td><a href="">Edit</a></td>
                <td><a class="delete" href="">Delete</a></td>
            </tr>
        </c:forEach>
    </table>
</c:if>
```

## Create 创建 

一个 CRUD 的 Create 需要 POST 请求，但是 POST 请求首先要求有一个基本的表单以供填写，然后对此表单执行 POST。因此需要为 Create 操作分别添加 GET 和 POST 两套方法， GET 用于从服务器请求需要填写的字段，然后提供表单页 POST 用于将此表单页上的内容在不重定向的基础上，进行 POST。

### Create GET

**提供表单基本信息:** 如果提供的字段需要按照一定的要求进行，比如有下拉列表，则同时要在 GET 方法返回 view 层的时候将这些必要的枚举对象一并传递到 request 域中。

**提供 FROM 更新的 POJO：** 如果使用 Spring 的 FORM 标签库，则需要提供一个用来设置的 POJO，FORM 标签库不会创建一个 POJO，我们需要从数据库或者构造器创建一个 POJO，然后传入。

```java
@RequestMapping(value = "emp",method = RequestMethod.GET)
public String addEmployeeGET(Map<String, Object> map) {
    map.put("department",departmentDao.getDepartments());
    map.put("newEmp", new Employee());
    return "add";
}
```

add.jsp 提供的实现如下，注意这里使用了 form 的标签库，这是 Spring MVC 提供的一个用于快速绑定数据对象的标签库。

```
<%
    Map<String,String> gender = new HashMap<>();
    gender.put("1","Male");
    gender.put("0","Female");
    //pageContext.setAttribute("gender",gender);
    request.setAttribute("genders",gender);
%>
<form:form action="" method="post" modelAttribute="newEmp" >    
    LastName: <form:input path="lastName" />
    Email: <form:input path="email" />
    Gender: <form:radiobuttons path="gender" items="${genders}" /><br>
    Department: <form:select path="department.id" items="${department}" itemLabel="departmentName" itemValue="id" /><br>
    Birth: <form:input path="birth" />
    Salary: <form:input path="salary" />
    <input type="submit" value="submit" />
</form:form>
```

需要注意，form 前缀标签用来从 modelAttribute 属性获得的 POJO 中填充字段。需要注意，我们必须为 form 标签提供一个 modelAttribute POJO 并且放置在 requestScope 中（也就是在 Mapping 的方法中从数据库或者构造器创建一个对象传入 ModelMap），否则会报错。

form 的 path 对应 pojo 的属性可选有 input 文字输入，radiobuttons 单选按钮，select 下拉按钮等。对于单选和下拉这种需要提供 key-value 的部件，从 items 传入 Map 以构造。或者传入 pojo 也可以，传入 pojo 需要提供 itemLabel 和 itemValue 分别对应的 pojo 属性 ,比如这里传入的是 department 变量，用它作为 select 的 iterator 来源，因此设置 `itemLabel="departmentName" itemValue="id"` 从每次遍历中获取 POJO 的属性填充 Select 下拉菜单。

注意，path 对应 modelAttribute 引用的 pojo 的属性，支持级联。

因为这里是 Restful Create，因此 action 指向相同的 URL，方法是 post。post 后会得到一个填充了 form 表单的 pojo。


### Create POST

调用 DAO 进行保存，之后重定向到 emps，注意，这里不是转发，而是重定向，因为没有逻辑关系。

POST 方法肯定会返回一个 Employee 对象，这里直接在入参处接受即可。注意，对于 GET 方法的入参，其含义为将此处的数据放置到 return 的 view 层的 request 域中。对于 POST 方法的入参，其不仅仅表示传递到下一个 view 的 request，还表示从 POST 源头接受一个对象。


```java
@RequestMapping(value = "emp", method = RequestMethod.POST)
public String addEmployeePOST(@ModelAttribute("newEmp") Employee employee, Map<String, Object> map)  {
    employeeDao.save(employee);
    return "redirect:emps";
}
```

## Delete 删除对象

删除操作是对于 `emp/{id}` 这种指定对象来说的。这里从 URL 中使用 @PathVariable 获取 id，然后调用 DAO 的 delete 方法进行删除，之后重定向到列表界面。注意：这里需要返回上级页面，以为这个 SpringMVC 对应的是 /crud 路径，因此不能使用根目录。要特别小心相对路径问题。

```java
@RequestMapping(value = "emp/{id}", method = RequestMethod.DELETE)
public String deleteEmployee(@PathVariable("id") Integer id) {
    employeeDao.delete(id);
    return "redirect:../emps";
}
```

删除的实现比较困难，原因在于浏览器基本上不支持 DELETE 的 HTTP 操作。我们需要使用 POST，然后内置一个隐藏方法来实现删除：

```html
<td><a class="delete" href="emp/${emp.id}">Delete</a></td>
```

问题是，一般的删除是 GET，那么如何在调用 GET emp/27 的时候自动调用 form 表单的 post，然后进行 delete 呢？使用 JQuery：

```html
<script src="https://cdnjs.cloudflare.com/ajax/libs/jquery/3.3.1/jquery.min.js"></script>
<script type="application/javascript">
    $(function() {
        $(".delete").click(function () {
                var href = $(this).attr("href");
                $(".delete_method").attr("action", href).submit();
                return false;
            }
        )
    })
</script>

<form class="delete_method" action="" method="post">
    <input type="hidden" name="_method" value="DELETE" />
</form>
```

当调用删除 GET 的时候，自动调用 FORM 的 POST(DELETE) 方法，将 GET 的 URL 传递给 FROM 的 ACTION，然后完成删除。注意，这里的 POST 使用了一个 hidden 的 input，此处的 name 必须是 `_method`, value 必须是 `DELETE`，之后必须为 TOMCAT 配置 HiddenMethodFilter 来过滤请求，之后才可以正常映射到 DELETE 方法的 Spring MVC Handler Mapping Method 上。

## Edit 更新对象

## Edit GET

因为我们 add 和 edit 用的是一个页面，因此 form 的 action 清空，默认为相同地址不同方法此外使用 if 标签判断 id，以确定是 add 还是 edit，如果是 edit，隐藏不允许修改字段，同时添加 `_method` 为 put，以进行修改而不是新建。

```java
@RequestMapping(value = "emp/{id}", method = RequestMethod.GET)
public String editEmployeeGET(@PathVariable("id") Integer id,
                              Map<String, Object> map){
    map.put("newEmp",employeeDao.get(id));
    map.put("department",departmentDao.getDepartments());
    return "add";
}
```

这里相比较 ADD，唯一的区别是，传入的不是空 Employee，而是从数据库获取的一个 Employee。

```
<c:if test="${newEmp.id == null}">
    LastName: <form:input path="lastName" />
</c:if>
<c:if test="${newEmp.id != null}">
    <form:hidden path="id" />
    <input type="hidden" name="_method" value="PUT" />
</c:if>
```

注意，这里的核心代码是，如果有 id（更新操作），那么新建一个隐藏的 input，用于指定此 POST 为 PUT。因为更新不能更新 id，所以使用 hidden 标签隐藏 ID 字段不进行更新。这里用来指定方法的标签是 input，而不是 form:input, 原因是 form name 必须在pojo 中，因此不能用 form:input。

注意，如果不希望更新某些字段，一定能够要使用 form:hidden，否则将会设置为 null。

### Edit PUT

表单默认调用同 Create POST - URL 的 POST 方法，不过这里因为有了隐含的 `_method` 指定为 PUT，因此经过过滤器过滤后变成了 PUT 方法，顺利匹配此处 Mapping 方法。

在方法中，将 FORM 提交的表单构造的 POJO 保存到 DAO，之后重定向到显示所有人的界面。

```java
@RequestMapping(value = "emp/{id}", method = RequestMethod.PUT)
public String editEmployeePUT(@ModelAttribute("newEmp") Employee employee){
    employeeDao.save(employee);
    return "redirect:../emps";
}
```

注意，这里的 Edit，我们不仅仅需要从 Form 表单中获取数据，而是需要从 Model 获取对象，然后由 Form 更新对象字段，然后返回，因此需要使用 @ModelAttribute：

```java
@ModelAttribute
public void getEmp(@RequestParam(value = "id", required = false) Integer id,
                     Map<String, Object> map) {
    if (id != null) {
        map.put("newEmp",employeeDao.get(id));
    }
}
```

这里尤其需要注意，因为 ADD 操作也会 POST 一个 Emp，因此也会自动寻找此 @ModelAttribute，然后试图更新此对象，而不是新建一个对象，最简单的方式是根据 id 区分，只对 Edit 的 POST 进行相应，而 Add，则自动创建一个新 POJO 即可。

# Spring MVC 数据绑定

数据绑定指的是从 FROM 表单到 POJO 对象的转换过程，在上面我们简要的介绍过基于 ModelAttribute 的服务器获取 - 表单更新 - 数据保存流程。在  CRUD 的示例中也进行了实际情况的展示。但是问题是，如果我们有级联属性，或者是想要对某个属性进行自定义的解析，然后进行保存，这时候就要处理数据类型转换的问题。此外，数据应该被格式化后进行保存，再者，数据需要进行校验，如果不满足需要，则需要回显数据，这些要求需要深入了解 MVC 的数据绑定机制。

我们说，数据绑定的核心问题有三，其一是数据类型转换，其二是数据格式化，其三是数据校验。

其中， ServletRequest 对象被传递给 WebDataBinderFactory 实例，之后创建了 DataBinder 实例对象。

DataBinder 对象得到需要处理方法的入参对象，准备对其进行绑定。

DataBinder 对象调用 ConversionService bean 进行数据类型的转换和数据格式化的共组，将 Servlet 中的请求信息填充到入参对象上。

调用 Validator 组件对于已经绑定了请求消息的入参对象进行数据合法性校验，并且最终生成数据绑定结果的 BindingData 对象，以及 BindingResult 对象。

总而言之，这些步骤-绑定、校验均以 DataBinder 作为主体， DB 接受 POJO 和 Request 作为材料，采用 ConversionService 和 Validator 作为工具进行转换，将 request 中的 form 表单转换并赋值给 POJO 的属性。

## ConversionService 转换服务

ConversionService 用于数据类型转换，一般而言，在调用注解驱动标签的时候，会自动添加默认的数据转换服务，请求映射服务，错误处理服务，基于注解的数据验证服务：

当启用了注解驱动的时候，会自动添加 RequestMappingHandlerAdapter，来解析注解处的 Mapping 请求。如果没有使用注解驱动，则仅仅有 SimpleControllerHandlerAdapter。注解驱动允许我们通过注解完成 URL 的映射和请求相应。此外， RequestMappingHandlerMapping、ExceptionHandlerExceptionResolver 也会在注解驱动时被自动添加到 IOC 中，以支持对于错误的处理。

此外，注解驱动将会为表单数据转换、格式化、验证提供支持：使用 conversion-service 属性以自定格式转换（需要提供一个包含了 set converters 的 CSFactoryBean）。注解驱动也为 @NumberFormat @DateTimeFormat 注解提供了格式化支持，为 @Valid 提供了验证支持

```xml
<mvc:annotation-driven conversion-service="formattingBean" />
<bean id="factoryBean"
      class="org.springframework.context.support.ConversionServiceFactoryBean">
    <property name="converters">
        <set><ref bean="converter"/></set>
    </property>
</bean>
<bean id="formattingBean"
      class="org.springframework.format.support.FormattingConversionServiceFactoryBean">
    <property name="converters">
        <set><ref bean="converter" /></set>
    </property>
</bean>
```

注意，最基本的自定义转换服务需要启用 `ConversionServiceFactoryBean`, 将其设置在 annotation-driven 的 conversion-service 标签中。对于需要除了数据类型转换，还需要启用基于注解的格式化的需求，则需要启用 `FormattingCnversionServiceFactoryBean` 此转换服务注册器。

这些注册器需要提供 converters 属性的注入，将我们自己实现的转换器作为 set 传入即可。

## 使用 Converter 接口实现自定类型转换

在配置中，我们已经将下面这个自定义转换器添加到了配置文件中，转换器代码如下：

```java
@Component
public class Converter implements org.springframework.core.convert.converter.Converter<String, Employee> {
    @Override
    public Employee convert(String s) {
        if (s != null) {
            String[] vals = s.split(":::");
            if (vals != null && vals.length == 4) {
                String lastName = vals[0];
                String email = vals[1];
                Integer gender = Integer.valueOf(vals[2]);
                Department department = new Department();
                department.setId(Integer.valueOf(vals[3]));
                Employee employee = new Employee(null,lastName,email,gender,department);
                System.out.println("Convert done!" + employee);
                return employee;
            }
        } return null;
    }
}
```

我们自己实现的 Converter 需要实现 `Converter<A,B>` 接口，其含义为当出现 A 类型时，将其转换为 B 类型，实现 convert 方法来进行转换。

下面是转换器的使用方法：

```html
<form action="/crud/testConverter" method="post">
    <input name="employee" type="text" />
    <input type="submit" name="submit" />
</form>
```

```java
@RequestMapping(value = "testConverter", method = RequestMethod.POST)
public String testConverter(@RequestParam("employee") Employee employee) {
    System.out.println(employee); employeeDao.save(employee);
    return "redirect:emps";
}
```

比如，输入： `Marvin:::cm@muninn.cn:::20:::10`，而在 Mapping 中需要一个 Employee 入参，则 SpringMVC 将会自动通过 Service 找到我们的转换器，调用 convert 进行转换。注意，自定义转换必须要求 SpringMVC 找到这两个类型，因此在入参处定义好 @RequestParam 的入参。注意，这里不是 @ModelAttribute 的入参，因为我们只想将 form 中的 employee 中获取数据，而不需要整个表单，表单提交时会将所有字段作为 Param 传入。


## 使用 ConverterFactory 进行多个 Converter 整合

除了使用 Converter 接口定义转换，还可以使用 ConverterFactory 将多个同质的 Converter 封装在一起，进行转换，比如 Number 以及 Number 的各个子类（Integer、Long、Double）。


## 使用 InitBinder 注解进行数据绑定设置

除了使用Converter，还可以对方法添加 @InitBinder 注解，此注解可以对 WebDataBinder 进行初始化，在由 FORM 到 POJO 绑定过程中起到核心作用。此注解一般使用 WebDataBinder 作为入参，不能有返回值。

在这里我们可以手动完成数据绑定，比如手工映射多个 radioBox 的 set 集合，或者使用 Editer 对象进行指定类型的数据转换`binder.registerCustomEditor(Class, Editor)`。这里等效于 Converter。

此外，还可以设置某个 POJO 属性不允许更改的字段 `setDisallowedFields`。

```java
@InitBinder
public void binder(WebDataBinder binder) {
    SimpleDateFormat dateFormat = new SimpleDateFormat("yyyy-MM-dd");
    dateFormat.setLenient(false);
    binder.registerCustomEditor(Date.class, new CustomDateEditor(dateFormat, false));*/
    binder.setDisallowedFields("lastName");
}
```

## 使用 FormattingCSFB 进行数据格式化

为了启用数据格式化支持，需要修改数据转换服务为 `FormattingConversionServiceFactoryBean`。如果在注解驱动的标签内什么也不写，默认是 `DefaultFormattingConversionService`。前者提供了对于数字和日期类型的支持：@NumberFormat, @DataTimeFormat

对于日期而言，可以对  Date、Calendar、Long 类型的时间标注，pattern 属性定义数据模式，iso 属性定义 iso 模式，style 属性定义字符串类型。

对于数字而言，pattern 指定数据模式，style 指定数字、货币还是百分数。

```java
@DateTimeFormat(pattern = "yyyy-MM-dd")
private Date birth;
@NumberFormat(pattern="#,###,###.#")
private Float salary;
```

## 使用 BindingResult 获取错误消息

如下，可以在获取的属性之后，紧挨着添加 BindingResult 类型的入参，其包含了 Employee 绑定的结果。其API 为：

Result.getErrirCount 可以获取错误条数。getFiledErrors 可以获取 FiledError 域映射错误列表，对其进行遍历，可以获得 域（getField）和错误信息（getDefaultMessage）。

```java
@RequestMapping(value = "emp/{id}", method = RequestMethod.PUT)
    public String editEmployeePUT(@ModelAttribute("newEmp") Employee employee,
                                  BindingResult result){
        if (result.getErrorCount() > 0) {
            System.out.println("出现错误!");
            for (FieldError error : result.getFieldErrors()) {
                System.out.println(error.getField() + ":" + error.getDefaultMessage());
            }
        }
    ...
}
```

此外，可以使用 Errors 对象而不使用 BindingResult 对象获取错误。当绑定出错的时候，可以回显进行显示，回显需要指定 `<form:error path='bean_attr' />` 来获取指定的域的错误信息，指定为 * 显示所有错误。如果出现错误的话，不提交，然后重新模拟 add 表单，不过现在已经自动填好了数据。

```java
@RequestMapping(value = "emp", method = RequestMethod.POST)
public String addEmployeePOST(@Valid @ModelAttribute("newEmp") Employee employee,
                              BindingResult result, Map<String, Object> map)  {
    if (result.getErrorCount() > 0) {
        System.out.println("出现错误!");
        for (FieldError error : result.getFieldErrors()) {
            System.out.println(error.getField() + ":" + error.getDefaultMessage());
        }
        map.put("department",departmentDao.getDepartments());
        return "add";
    }
    employeeDao.save(employee);
    return "redirect:emps";
}
```

## 使用 JSR303 进行数据校验

JSR303 提供了验证标准，Hibernate validator 提供了验证框架，需要启用注解驱动的标签，然后自动会配置验证框架的 jar 包，在 bean 属性添加对应验证注解，在需要验证的入参处提供 @Valied 注解，即可完成验证，当出现错误的时候，默认放置在 BindingResult 中。

```java
@NotEmpty private String lastName;
@Email private String email;
@DateTimeFormat(pattern = "yyyy-MM-dd") @Past private Date birth;
@NumberFormat(pattern="#,###,###.#") private Float salary;
```

验证的类属于 javax.validation.constraints。

此外还有 Null，必须为 null；NotNull 不为 null；AssertTrue/False 必须为 true/false；Min/Max 最大最小值；DecimalMin/Max 必须为数字，最小最大为多少；Size(a,b) 范围；Digits(a,b) 数字，在范围内；Past 过去日期；Future 将来日期；Pattern 满足正则表达式。

```java
@RequestMapping(value = "emp", method = RequestMethod.POST)
    public String addEmployeePOST(@Valid @ModelAttribute("newEmp") Employee employee,
                                  BindingResult result, Map<String, Object> map)  { ...  
}
```

## 提示消息的国际化

同样的，需要先配置 messageSource，指定 i18n 文件名，作为属性传递进去。之后，定义各种注解情况下的错误翻译信息：

```
NotEmpty.employee.lastName=xxx
Email.employee.email=xxx
```
格式为 `注解.bean的引用名.属性`。这样的话，制定名称的 bean 发生指定注解的错误后，会使用此处作为错误消息显示。

除了注解，内部错误，需要参数，类型不匹配都可以进行消息国际化：

```
methodInvocation.xx.xxx 表明 SpringMVC 在调用处理方法时发生了错误
required.xx.xxx 需要某参数
typeMismatch.xx.xxx 类型不匹配
```

# HttpMessageConverter 输入输出转换

## POJO 集合直接输出 JSON 数据

毫无意外的，我们需要 Jackson 的 jar 包。此外，只需要为 Mapping 方法添加 @RequestBody 注解即可。注意，下面的方法返回的是一个 Collection 泛型的集合对象，添加注解后，自动被解析成了 Json 文件。

```java
@ResponseBody @RequestMapping("testJson")
public Collection<Employee> testJson() {
    return employeeDao.getAll();
}
```

在这个过程中，进行转换的核心类是 `HttpMessageConverter<T>`。

请求报文首先被转换为 HttpInputMessage，然后被传入 HttpMessageConverter，之后变为 Java 对象交给 MVC 进行处理。反过来，MVC 交给 HttpMessageConverter Java 对象，然后此对象被其转换成 HttpOutputMessage，然后作为响应报文。HttpInputMessage 只有一个方法，获取请求的 InputStream，然后转给 HttpMessageConverter。反之也一样（HttpOutputMessage）。

Spring 内置了一些 messageConverter，jackson 包加入后会自动启用此 converter。

具体使用如下：@RequestBody（传入值处理） @ResponseBody（返回值处理），具体使用哪种 converter，取决于泛型/注解的类型。

## 基于 HMC 的上传和下载

下面这个方法，我们在入参使用了 @RequestBody 注解，此注解将会将此方法调用的所有信息传入 String 类型的 body 变量中，也就是我们在这里上传的文件。注意到，此处的 @RequestBody 进行了输入二进制流和 String 之间的转换。

```html
<form action="testUpload" method="post" enctype="multipart/form-data">
    Files: <input type="file" name="file" />
    Desc: <input type="text" name="desc" />
    <input type="submit" value="submit" />
</form>
```

```java
@ResponseBody @RequestMapping("testUpload")
public String testUpload(@RequestBody String body) {
    System.out.println(body);
    return "helloworld" + new Date();
}
```

因此，通过打印，可以看到这个上传的文件信息。

此外，还可以使用 @ResponseBody 注解处理回传的信息。默认情况下，返回字符串会去寻找 ViewResolver 和 View，完成渲染，这里返回的字符串不会通过 Mapping 去寻找 jsp，也不会去寻找静态资源，因为添加了 @ResponseBody 注解，因此直接按照 String 找到对应的 StringConverter 返回字符串。

注意，这里也不是直接向 out.write 直接打印，而是经过了一些包装的。这就是 @ResponseBody 对于输出进行的转换。

转换选用哪个 Converter，取决于参数类型（对于注解而言），或者泛型（对于 `HttpEntity<T> 和 ResponseEntity<T>`）

下面的例子介绍了使用基于泛型的转换方式，这种方式需要提供数据转换的泛型信息。

```java
@RequestMapping("testDownload")
public ResponseEntity<byte[]> testDownload(HttpSession session) throws IOException {
    byte[] body = null;
    ServletContext context = session.getServletContext();
    InputStream in = context.getResourceAsStream("/files/mongo.txt");
    body = new byte[in.available()];
    in.read(body);

    HttpHeaders headers = new HttpHeaders();
    headers.add("Content-Disposition","attachment;filename=mongo.txt");

    HttpStatus statusCode = HttpStatus.OK;

    ResponseEntity<byte[]> entity = new ResponseEntity<>(body, headers, statusCode);
    return entity;
}
```

我们在这里由 session 得到了 ServletContext，然后由此 Context 得到了磁盘文件，然后作为输入流，放入 ResponseEntity 中，添加 Header 和 HttoStatus，进行返回，完成了简易的动态下载功能。

## 基于 MultipartResolver 的文件上传

SpringMVC 通过 MultipartResolver 提供对于上传文件的支持，相比较我们使用 HMC 进行简单的上传下载，MR 更加强大。默认的 MR 是基于 Jakarta Commons FileUpload 实现的 CommonsMultipartResolver，需要手动配置此 bean。

```xml
<bean class="org.springframework.web.multipart.commons.CommonsMultipartResolver" 
      id="multipartResolver" p:defaultEncoding="UTF-8" p:maxUploadSize="5242880" />
```

注意，此处的 id 必须为 multipartResolver，否则出错。

```html
<form action="testUpload" method="post" enctype="multipart/form-data">
    文件 : <input type="file" name="file"/><br><br>
    描述 : <input type="text" name="desc"/><br><br>
<input type="submit" value="提交 "/>
</form>
```

```java
@RequestMapping("testMUpload")
public String testMUpload(@RequestParam("file") MultipartFile file) throws IOException {
    InputStream inputStream = file.getInputStream();
    return "success";
}
```

# Spring MVC 国际化

同样的，声明 i18n.properties 文件，提供 messageSourse 连接此文件，使用 fmt:message 来显示国际化标签。

我们可以在 Java 类中获取国际化信息，需要先配置 ResourceBundleMessageSource，这就是我们配置的 mS bean，直接注入就好。在 Handler 中直接传入 Locale 参数，即可获取地区信息，使用 ms.getMessage 获取指定字段在指定 locale 的叫法。

```java
private final ResourceBundleMessageSource messageSource;

    @Autowired
    public MoreTest(ResourceBundleMessageSource messageSource) {
        this.messageSource = messageSource;
    }

    @RequestMapping("i18n")
    public String sayHello(Locale locale) {
        String val = messageSource.getMessage("i18n.username",null,locale);
        System.out.println(val);
        return "i18n";
    }
}
```

此外，我们可以通过 `i18n2?locale=zh_CN` 这样的超链接切换地区。其实现原理为：

默认情况下，是通过 Accept-Language 来判断 Local 类型的，当接受一个 Http 请求以后，SMVC 自动查找 LocalResolver，然后用它获得本地化的类型信息。我们可以通过装配一个动态更改本地化类型的拦截器拦截本地化调用，称之为 LocaleChangeInterceptor，此拦截器将会拦击 Locale 对象的创建，然后使用替代的 Locale 对象，之后通过它获取 LocalResolver 对象（SessionLocaleResolver），SLR 之后将自动 设置 Locale 为 Session 的属性，然后应用程序就会使用此 Locale 对象。

为了进行拦截和使用新的 SLR，我们需要进行配置:

```xml
<!--配置基于Session的位置解析器和位置切换截获器-->
<mvc:interceptors>
    <bean class="org.springframework.web.servlet.i18n.LocaleChangeInterceptor"/>
</mvc:interceptors>
<bean class="org.springframework.web.servlet.i18n.SessionLocaleResolver" id="localeResolver"/>
```

# Spring MVC 拦截器

## HandlerInterceptor API

拦截器需要手动在 xml 文件中添加 `<mvc:interceptors> <bean class="com.mazhangjing.mvc.more.FirstInterceptor" /> </mvc:interceptors>`才可使用。拦截器需要实现三个方法：preHandle，postHandle，afterCompletion。

preHandle 在目标方法请求前执行，如果返回 false，则不会调用后续其余的拦截器和目标方法（业务处理器）。如果是 true，则正常执行在调用目标方法之前调用，可以用来做权限和日志、事务等。

postHandle 在调用目标方法（HandlerAdapter.handle）之后，但在渲染视图之前调用此方法可以用来对请求域中的属性或视图做出修改。

afterCompletion 在完成视图渲染( DispatcherServlet.render)之后调用。进行一些清理工作。

```java
public class FirstInterceptor implements HandlerInterceptor {
    @Override
    public boolean preHandle(HttpServletRequest httpServletRequest, HttpServletResponse httpServletResponse, Object o) {
        System.out.println("preHandle");
        return true;
    }

    @Override
    public void postHandle(HttpServletRequest httpServletRequest, 
                           HttpServletResponse httpServletResponse, Object o, ModelAndView modelAndView) {
        System.out.println("postHandle");
    }

    @Override
    public void afterCompletion(HttpServletRequest httpServletRequest, 
                                HttpServletResponse httpServletResponse, Object o, Exception e) {
        System.out.println("afterHandle");
    }
}
```

## 配置 HandlerInterceptor

除了配置能共应用于所有 URL 请求的拦截器，还可以使用 interceptor 标签配置只作用/不作用于某路径的拦截器：

```xml
<mvc:interceptors>
    <bean class="com.mazhangjing.mvc.more.FirstInterceptor" />
    <mvc:interceptor>
        <mvc:mapping path="/more/sec"/>
        <bean class="com.mazhangjing.mvc.more.SecondInterceptor" />
    </mvc:interceptor>
</mvc:interceptors>
```

多个 Interceptor 执行的顺序是，在三个阶段（HandlerAdaptor 之前、HandlerAdaptor - DispatcherServlet、DispatcherServlet 之后），分别是1-2,2-1,2-1执行。

如图：

![](interceptor.png)

注意，如果第二个拦截器返回 false，则会跳转到第一个拦截器，执行资源释放擦操作（实线部分）。

# Spring MVC 异常处理

默认在 mvc 注解驱动的设置中，配置了异常处理类，如果没有显式指定，则 HandlerExceptionResolver 默认配置了 Exeption、ResponseStatus、Defalut 三个 ER。如果没有使用注解驱动，则配置了 AnnotationMethod、ResponseStatus、Defalut ER。

## ExceptionHandlerExceptionResolver

EHResolver 负责解析 ExceptionHandler 提供的异常处理，然后进行响应：

```java
@RequestMapping("handle")
public String handleEx(@RequestParam("i") int i){
    System.out.println("result is " + 10/i);
    return "hello";
}

@ExceptionHandler({ArithmeticException.class})
public ModelAndView handleArithmeticException(Exception ex) {
    System.out.println("ex = " + ex);
    ModelAndView error = new ModelAndView("error");
    error.addObject("err_info",ex);
    return error;
}
```

当出现异常，自动去 Handler 找有 @ExceptionHandler 注解的方法，如果匹配有此注解，且异常类型匹配的方法，那么进入执行。此方法可传入 Exception 参数，用以接受错误信息。但是不可传入 Map 参数，向 view 层传递数据。为了展示 错误界面，应该返回 ModelAndView 对象，手动调用 addObject 来放入错误信息，传递给 view 层。

对于注解，定义越详细的注解，优先级越高，会拿来进行相应，而忽略泛化那个（如果我们还有一个 RuntimeException 的 Handler 的话，RE 会被忽略）。

此外，如果在 Handler 无法找到 ExceptionHandler 的时候，去寻找类路径下任意有 ControllerAdvice 注解的异常处理专用类，此注解会自动添加 beans 到 IOC。如下：

```java
@ControllerAdvice
public class HandleException {

    @ExceptionHandler({ArithmeticException.class})
    public ModelAndView handleArithmeticException(Exception ex) {
        System.out.println("ex = " + ex);
        ModelAndView error = new ModelAndView("error");
        error.addObject("err_info",ex);
        return error;
    }
}
```

可以在此类中统一完成错误的处理，而不用分布在各个 Mapping 的 Controller 中。

## ResponseStatusExceptionResolver

对于我们自己的异常类，可以添加 @ResponseStatus 注解，提供 value 和 reason，当出现异常，将会返回此 HTTP 状态码，并写入此 reason 信息。

```java
//Handler.java
@RequestMapping("handleS")
public String handleStatusEx(@RequestParam("i") int i){
    if (i == 13) throw new IDoNotCareException();
    return "hello";
}
//IDoNotCareException.java
@ResponseStatus(value = HttpStatus.FORBIDDEN, reason = "I don't care who you are")
public class IDoNotCareException extends RuntimeException {
    ...
}
```

注意，使用 value 提供返回状态码，使用 reason 提供原因。执行访问 `http://localhost:8080/more/handleS?i=13`，异常结果为：HTTP 403。

此外，还可以对任意的类方法添加此注解，造成的结果就是，原本没有错误的 Handler 也不能正常 Mapping，直接返回此错误代码。用于直接对某些 URL 访问不经处理，直接返回 HTTP 代码的情况。

```java
@RequestMapping(value = "/go",params = {"name=trump"})
@ResponseStatus(value = HttpStatus.CONFLICT, reason = "Conflict!!!")
public void handleSomething() { }
```

访问此特定 URL（传入此 Param），则直接返回错误。

## DefaultHandlerExceptionResolver

此异常解释器对于 Spring 特殊的异常进行处理，比如 GET 请求只有 POST 的 Mapping 这种情况。自动进行映射并且返回 Http 的异常。

## SimpleMappingExceptionResolver

在 ExceptionHandlerExceptionResolver 的时候，如果出现异常，我们需要提供一个异常处理器，如果要转到 view 层，那么还需要手动构建 ModelAndView 对象，这样返回的页面是一个纯粹的错误页面。但是，有时候我们发现仅仅出了一点错误，希望正常返回，但是顺便把错误包含在这个页面上。

对于 SimpleMappingExceptionResolver 而言，我们可以通过 XML 配置文件声明异常和其被注入的页面的 attribute。之后，在原来的 Mapping 中，直接返回正常页面，错误通过其页面的 exception（或者在 XML 定义的其他 exceptionAttribute 值）来获取。

```java
@RequestMapping("handleM")
public String doHandlerMapping(@RequestParam("i") int i){
    String[] vals = new String[10];
    System.out.println("vals = " + vals[i]);
    return "result";
}
```

```xml
<bean class="org.springframework.web.servlet.handler.SimpleMappingExceptionResolver">
    <property name="exceptionAttribute" value="aexception" />
    <property name="exceptionMappings">
        <props>
            <prop key="java.lang.ArrayIndexOutOfBoundsException" />
        </props>
    </property>
</bean>
```

```
/result.jsp
<c:if test="${aexception ne null}">
You may have a problem: ${aexception}
</c:if>
```

# Spring MVC 运行流程

## 从请求到响应

![](spring-mvc.png)

## Spring MVC 整合 Spring

比如 Hibernate 框架的 SessionFactory，或者其被包裹的 Data Access Object、包裹 DAO 的 Service，这些都应该放在 Spring 的 IOC 容器中。而不应该被放在 Spring MVC 的配置文件或者导入其配置文件中。

为了配合工作，必须在 Web 服务器启动的时候加载 Spring IOC，采用监听器完成：

```xml
<context-param>
    <param-name>contextConfigLocation</param-name>
    <param-value>classpath:beans.xml</param-value>
</context-param>
<listener>
    <listener-class>org.springframework.web.context.ContextLoaderListener</listener-class>
</listener>
```

需要将配置加载到 web app 的 contextConfigLocation 参数中，提供 Spring 的配置文件，使用 ContextLoaderListener 来进行 IOC 加载。

因为 Spring 和 MVC 都会进行包扫描，因此需要设置 MVC 只扫描指定注解：

```xml
<context:component-scan base-package="com.mazhangjing.mvc.more" use-default-filters="false">
    <context:include-filter type="annotation" expression="org.springframework.stereotype.Controller" />
    <context:include-filter type="annotation" expression="org.springframework.web.bind.annotation.ControllerAdvice" />
</context:component-scan>
```

注意，需要使用注解类，此外，禁用默认过滤器。在 Spring 配置文件中 exclude 这两个注解。

尤其需要注意，Spring 不能够使用 Spring MVC IOC 容器的 bean，但是 Spring MVC IOC 却可以引用 Spring 的 bean。其实 MVC 的 IOC 是 Spring IOC 的一个子容器，所有 MVC 的 bean 属于局部作用域，可以以来 Spring 的外部作用域，但是反向则不行。